### Memory 최적화 이전 버전
- 모든 정보를 램에 올려두고 사용

In [1]:
# 효율적인 gpu 사용
import tensorflow as tf
from keras.backend import tensorflow_backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.set_session(tf.Session(config=config))

Using TensorFlow backend.


### 폴더 내 모든 파일 리스트 읽기

In [2]:
import os
from PIL import Image
import numpy as np

img_idx_stride = 30 # 이미지 번호가 뛰는 단위. 00000 -> 00030 ...
look = 0
scaler = 3
scaled = (1640//scaler + 1, 590//scaler)
crop_x = 1
crop_y = 2

In [3]:
def read_set(target):
    '''
    # Input
    target : 뒤져볼 폴더명
    
    # Output
    imgs, labels = 데이터와 정답. numpy 형식
    '''
    print('Start to read {} and {}'.format(target+'/data', target+'/label'))
    
    # Train data Read
    imgs = []

    ## data Read
    for root, dirs, files in os.walk(target+'/data'):
        # 일정 순서대로 읽기
        dirs.sort()
        files.sort()
        for fname in files:
            full_fname = os.path.join(root, fname)
            tmp_img = Image.open(full_fname)
            tmp_arr = np.array(tmp_img.resize(scaled))
            imgs.append(tmp_arr)
            del(tmp_img)
            del(tmp_arr)

    ##label Read
    labels = []

    for root, dirs, files in os.walk(target+'/label'):
        # 일정 순서대로 읽기
        dirs.sort()
        files.sort()
        for fname in files:
            full_fname = os.path.join(root, fname)
            tmp_img = Image.open(full_fname)
            tmp_arr = np.array(tmp_img.resize(scaled))[:-crop_y,:-crop_x]
            labels.append(tmp_arr)
            del(tmp_img)
            del(tmp_arr)

    X = np.array(imgs)
    y = np.array(labels)
    
    del(labels)
    del(imgs)
    
    return X, y

    # print("X_train has {} shape, y_train has {} shape".format(X_train.shape, y_train.shape))

    # Make Global variables
    #total, width, height = X_train.shape[0], X_train.shape[1], X_train.shape[2]

In [4]:
# Train data Read
X_train, y_train = read_set('train')
print("X_train has {} shape, y_train has {} shape".format(X_train.shape, y_train.shape))

# Test data Read
X_test, y_test = read_set('test')
print("X_test has {} shape, y_test has {} shape".format(X_test.shape, y_test.shape))

# Make Global variables
train_size, test_size , width, height = X_train.shape[0], X_test.shape[0], X_train.shape[1], X_train.shape[2]

Start to read train/data and train/label
X_train has (17790, 196, 547, 3) shape, y_train has (17790, 194, 546) shape
Start to read test/data and test/label
X_test has (3357, 196, 547, 3) shape, y_test has (3357, 194, 546) shape


In [ ]:
# Check data

import matplotlib.pyplot as plt
rnum = np.random.randint(X_train.shape[0])

plt.figure(figsize=(40,4))
plt.title("Input")
plt.imshow(X_train[rnum])
plt.show()

plt.figure(figsize=(40,4))
plt.title("Output")
plt.imshow(y_train[rnum],cmap='gray')
plt.show()

### Data 가공
- LSTM에 맞게, 시간 순서 구조를 부여한다. (데이터 수, 가로, 세로) -> (데이터 수, 현재 보고 있는 데이터들, 가로, 세로)
- 한 번에 3개의 데이터를 보고, 하나의 데이터를 생성한다.

#### Data 정보
- Training set : [CULane](https://xingangpan.github.io/projects/CULane.html) - driver_182_30frame.tar.gz, 5.5GB
- Validation set : [CULane](https://xingangpan.github.io/projects/CULane.html) - driver_37_30frame.tar.gz, 1GB

In [ ]:
def give_time(data_size, X, y, memory_size = 3):
    # Make time-dependent data
    # (data_idx, x, y) -> (data_idx, looking, x, y)
    X_t = np.zeros((data_size, memory_size, width, height, 3),dtype='uint8')
    y_t = np.expand_dims(y, axis = 3)

    for i,e in enumerate(X):
        try:
            X_t[i] = X[i:i+memory_size]
        except:
            print('* Error : stop at idx {}'.format(i))
            break
    
    return X_t, y_t

## 램 모자란다!!
- 램 증가 (연구실에 있는거 동원)
- 블럭 단위로 읽어오기 추가

In [ ]:
# 저장할 기간을 정한다.
memory_size = 3
input_shape = X_train.shape[1:]

X_train_t, y_train_t = give_time(train_size, X_train, y_train, memory_size)
del(X_train)
del(y_train)

X_test_t, y_test_t = give_time(test_size, X_test, y_test, memory_size)
del(X_test)
del(y_test)

import gc
gc.collect()

In [ ]:
# Check time-dependency is correct
for i in range(memory_size):
    plt.figure(figsize=(40,4))
    plt.imshow(X_train_t[rnum, i])
plt.show()

### Neural Network 정의

In [ ]:
print(input_shape)

In [ ]:
# Batch size, epochs and pool size below are all paramaters to fiddle with for optimization
batch_size = 10
epochs = 3
pool_size = (2, 2)
input_shape = X_train_t.shape[1:]

In [ ]:
# Import necessary items from Keras
from keras.models import Sequential
from keras.layers import Activation, Dropout, UpSampling2D
from keras.layers import Conv2DTranspose, Conv2D, MaxPooling2D, ConvLSTM2D, TimeDistributed
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

### Here is the actual neural network ###
model = Sequential()
# Normalizes incoming inputs. First layer needs the input shape to work
model.add(BatchNormalization(input_shape=input_shape))

# Below layers were re-named for easier reading of model summary; this not necessary
# LSTM Conv Layer 1
model.add(ConvLSTM2D(filters=8, kernel_size=(3, 3),strides=(1,1), data_format='channels_last',
                     padding='valid', return_sequences=True))

# LSTM Conv Layer 2
model.add(ConvLSTM2D(filters=16, kernel_size=(3, 3),strides=(1,1), data_format='channels_last',
                     padding='valid'))

# Pooling 1
model.add(MaxPooling2D(pool_size=pool_size))

# Conv Layer 3
model.add(Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv3'))
model.add(Dropout(0.2))

# Conv Layer 4
model.add(Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv4'))
model.add(Dropout(0.2))

# Conv Layer 5
model.add(Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv5'))
model.add(Dropout(0.2))

# Pooling 2
model.add(MaxPooling2D(pool_size=pool_size))

# Conv Layer 6
model.add(Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv6'))
model.add(Dropout(0.2))

# Conv Layer 7
model.add(Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv7'))
model.add(Dropout(0.2))

# Pooling 3
model.add(MaxPooling2D(pool_size=pool_size))

# Upsample 1
model.add(UpSampling2D(size=pool_size))

# Deconv 1
model.add(Conv2DTranspose(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv1'))
model.add(Dropout(0.2))

# Deconv 2
model.add(Conv2DTranspose(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv2'))
model.add(Dropout(0.2))

# Upsample 2
model.add(UpSampling2D(size=pool_size))

# Deconv 3
model.add(Conv2DTranspose(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv3'))
model.add(Dropout(0.2))

# Deconv 4
model.add(Conv2DTranspose(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv4'))
model.add(Dropout(0.2))

# Deconv 5
model.add(Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv5'))
model.add(Dropout(0.2))

# Upsample 3
model.add(UpSampling2D(size=pool_size))

# Deconv 6
model.add(Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv6'))

# Deconv 7
model.add(Conv2DTranspose(8, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv7'))

# Final layer - only including one channel so 1 filter
model.add(Conv2DTranspose(1, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Final'))

In [ ]:
model.summary()

In [ ]:
# Compiling and training the model
model.compile(optimizer='Adam', loss='mean_squared_error', metrics=['accuracy'])
hist = model.fit(x=X_train_t, y=y_train_t, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test_t, y_test_t))

### 정확도 및 손실율 추산

In [ ]:
import matplotlib.pyplot as plt

## summarize history for accuracy
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## summarize history for loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#[출처] 케라스(Keras) 기본 - 모델 학습, 히스토리 기능, 모델(신경망) 생성, 시각화 등|작성자 예비개발자

In [ ]:
print("Final accuracy is {:.3f}%".format(hist.history['val_acc'][-1]*100))

# Save model architecture and weights
model.save('tanuki_network.h5')

import gc
gc.collect()